In [1]:
import dash
from dash.dependencies import Output, Event, Input
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque
import sqlite3
import pandas as pd
import time
import numpy as np

In [ ]:
#popular topics: google, olympics, trump, gun, usa


pos_neg = deque(maxlen=1)

app = dash.Dash(__name__)
app.layout = html.Div(
    [   html.H2('ProjectX - Live Twitter Sentiment'),
        dcc.Input(id='sentiment_term', value='Trump', type='text'),
        dcc.Graph(id='live-graph', animate=False),
        dcc.Interval(
            id='graph-update',
            interval=0.5*1000
        ),
     dcc.Graph(id='sentiment-pie', animate=False),
     dcc.Interval(
             id='sentiment-pie-update',
             interval=5*1000
     ),
     
    ],
    
)


@app.callback(Output('live-graph', 'figure'),
              [Input(component_id='sentiment_term', component_property='value')],
              events=[Event('graph-update', 'interval')])
def update_graph_scatter(sentiment_term):
    try:
        conn = sqlite3.connect('twitter.db')
        c = conn.cursor()
        df = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE ? ORDER BY unix DESC LIMIT 1000", conn ,params=('%' + sentiment_term + '%',))
        df.sort_values('unix', inplace=True)
        df['sentiment_smoothed'] = df['sentiment'].rolling(int(len(df)/2)).mean()
        
        pos = df['sentiment_smoothed'].gt(0).sum()
        neg = df['sentiment_smoothed'].lt(0).sum()
        
        pos_neg.append([pos,neg])
        
        df['date'] = pd.to_datetime(df['unix'],unit='ms')
        df.set_index('date', inplace=True)
        
        df = df.resample('1s').mean()
        df.dropna(inplace=True)
        X = df.index
        Y = df.sentiment_smoothed
        
        data = plotly.graph_objs.Scatter(
                x=X,
                y=Y,
                name='Scatter',
                mode= 'lines+markers'
                )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),
                                                    yaxis=dict(range=[min(Y),max(Y)]),
                                                    title='Term: {}'.format(sentiment_term))}

    except Exception as e:
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')

            
@app.callback(Output('sentiment-pie', 'figure'),
              [Input(component_id='sentiment_term', component_property='value')],
              events=[Event('sentiment-pie-update', 'interval')])
def update_pie_chart(sentiment_term): 
    try:
        conn = sqlite3.connect('twitter.db')
        c = conn.cursor()
        df2 = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE ? ORDER BY unix DESC LIMIT 10000", conn ,params=('%' + sentiment_term + '%',))
        df2['sentiment_smoothed'] = df2['sentiment'].rolling(int(len(df2)/2)).mean()
        
        pos = df2['sentiment_smoothed'].gt(0).sum()
        neg = df2['sentiment_smoothed'].lt(0).sum()
        
        #pos = pos_neg[0][0]
        #neg = pos_neg[0][1]
        
        labels = ['Positive','Negative']

        
        values = [pos/(pos+neg),neg/(pos+neg)]
        
        print(pos,neg,values)
        colors = ['#007F25', '#800000']

        trace = go.Pie(labels=labels, values=values,
                       hoverinfo='label+percent', textinfo='value', 
                       textfont=dict(size=20),
                       marker=dict(colors=colors))
        
        return {"data":[trace],'layout' : go.Layout(
            title='Positive vs Negative sentiment for "{}" (longe term)'.format(sentiment_term),
            showlegend=True)}
    
    except Exception as e:
        print(e)
        #with open('errors_pie.txt','a') as f:
        #    f.write(str(e))
        #    f.write('\n')

            
            
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


68

127.0.0.1 - - [13/Nov/2018 23:03:41] "POST /_dash-update-component HTTP/1.1" 200 -


 1 [0.98550724637681164, 0.014492753623188406]


...

127.0.0.1 - - [13/Nov/2018 23:16:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 23:16:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 23:16:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 23:16:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 23:16:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 23:16:40] "POST /_dash-update-component HTTP/1.1" 200 -


49 22 [0.6901408450704225, 0.30985915492957744]


127.0.0.1 - - [13/Nov/2018 23:16:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 23:16:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 23:16:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 23:16:43] "POST /_dash-update-component HTTP/1.1" 200 -


49 22 [0.6901408450704225, 0.30985915492957744]


127.0.0.1 - - [13/Nov/2018 23:16:43] "POST /_dash-update-component HTTP/1.1" 200 -
